# Оценка RAG с помощью разных метрик 

In [1]:
import os
from necessity import *

## Загрузка Датасета c ответами

In [2]:
import pandas as pd

# встаивть путь датасета для проверки
path = '../rag_answer/base_rag/base_rag_gpt.csv'
path2 = '../all_ratings/eval_base_rag'
df = pd.read_csv(path)
display(df)

,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,"Да, можно не расписываться на карте при возвра..."
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...","Вам следует быть осторожным, если ваш друг пр..."
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно хранить ПИН-код в заметках в..."
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...","Свяжитесь с банком, который установил банкомат..."
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,При использовании карты важно не расписываться...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста..."
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...","Нет, не рекомендуется использовать интернет-ка..."
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,1. Поставьте свою подпись на оборотной стороне...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...",При бронировании гостиницы по карте необходимо...


In [3]:
df['contexts'] = df['contexts'].apply(lambda x:[x])

In [4]:
import datasets
test_dataset = datasets.Dataset.from_dict(df)
my_dataset_dict = datasets.DatasetDict({"Evaluete":test_dataset})

/Users/anastasia/Desktop/ML/Диплом/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
my_dataset_dict

DatasetDict({
    Evaluete: Dataset({
        features: ['question', 'contexts', 'ground_truth', 'answer'],
        num_rows: 58
    })
})

## Оценка метрик с помощью RAGAS

In [8]:
from ragas.metrics import (
    answer_relevancy,
        faithfulness,
        answer_relevancy,
        context_precision,
        context_relevancy,
        answer_similarity,
)

embedding_model = OpenAIEmbeddings()
llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125")

In [9]:
from ragas import evaluate

result = evaluate(
    test_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_relevancy,
        answer_similarity
    ],
    llm=llm,
    embeddings=embedding_model
)

Evaluating:  48%|████▊     | 139/290 [00:41<00:36,  4.19it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  69%|██████▉   | 201/290 [00:58<00:22,  3.89it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  82%|████████▏ | 237/290 [01:09<00:12,  4.27it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  91%|█████████ | 264/290 [01:15<00:05,  4.76it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating: 100%|██████████| 290/290 [01:31<00:00,  3.17it/s]


In [10]:
result

{'faithfulness': 0.9435, 'answer_relevancy': 0.8193, 'context_precision': 1.0000, 'context_relevancy': 0.7500, 'answer_similarity': 0.9237}

In [94]:
# # Открытие файла для записи ('w' - режим записи)
# with open(path2 + '/metrics.txt', 'w') as file:
#     file.write(str(result))

In [11]:
df = result.to_pandas()
df

,question,contexts,ground_truth,answer,faithfulness,answer_relevancy,context_precision,context_relevancy,answer_similarity
0,Можно ли не расписываться на карте?,[['После получения карты поставьте свою подпис...,При отсутствии подписи карта может быть не при...,"Да, можно не расписываться на карте при возвра...",NaN,0.950380,1.0,1.000000,0.834871
1,Мой друг попросил меня дать свою карту для опл...,"[['Никогда не принимайте рекомендации, советы,...","Не передавайте карту другим лицам, в т.ч. родс...","Вам следует быть осторожным, если ваш друг пр...",NaN,0.000000,1.0,1.000000,0.899626
2,Безопасно ли хранить пинкод в заметках в телеф...,[['Категорически запрещается записывать ПИН-ко...,"Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно хранить ПИН-код в заметках в...",NaN,0.962787,1.0,0.000000,0.890383
3,Я потеряла карту,[[' Вы потеряли карту или ее украли\nПрежде вс...,"Если вы не уверены, что карту украли, не спеши...","Свяжитесь с банком, который установил банкомат...",NaN,0.805542,1.0,0.100000,0.849157
4,Почему при использовании карты важно не распис...,"[['Проверяйте правильность суммы, указанной на...",Важно проверять правильность суммы перед подпи...,При использовании карты важно не расписываться...,NaN,0.940811,1.0,1.000000,0.919985
5,"Что делать, если я потеряла телефон, на которо...","[['При утрате Мобильного телефона, используемо...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста...",NaN,0.979307,1.0,1.000000,0.973052
6,Можно ли использовать интернет-кафе и другие о...,[['Избегайте осуществления Интернет-операций с...,"Нет, не рекомендуется использовать интернет-ка...","Нет, не рекомендуется использовать интернет-ка...",NaN,0.000000,1.0,1.000000,0.941810
7,"Что делать, если я получил SMS-оповещение о оп...","[['Если Вы уверены, что операция не могла быть...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...,1.000000,0.977498,1.0,1.000000,0.960059
8,Как обезопасить себя от съема данных карты в б...,"[['Поэтому, при пользовании банкоматом не лени...",Чтобы обезопасить себя от съема данных карты в...,1. Поставьте свою подпись на оборотной стороне...,NaN,0.842898,1.0,1.000000,0.891949
9,"Правда ли, что при бронировании гостиницы по к...",[['При бронировании номера в гостинице по карт...,"Нет, неправда. При бронировании гостиницы по к...",При бронировании гостиницы по карте необходимо...,NaN,0.895663,1.0,1.000000,0.957098


## Другие метрики

### Оценка с помощью bleu_score

In [12]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(row):
    reference = row['answer'].split()           # Токенизация ответа llm
    candidate = row['ground_truth'].split()     # Токенизация гипотезы
    return sentence_bleu([reference], candidate)

# Применение функции к DataFrame
df['bleu_score'] = df.apply(calculate_bleu, axis=1)
df['bleu_score'] = df['bleu_score'].apply(lambda x: round(x, 4))

/Users/anastasia/Desktop/ML/Диплом/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/anastasia/Desktop/ML/Диплом/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/anastasia/Desktop/ML/Диплом/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order i

### Оценка с косинусным сходством 


Оба метода используют косинусное сходство для оценки схожести между текстами, однако различаются в подходе к векторизации текста.

**Первый метод**:
- Использует метод TF-IDF для векторизации текстов.
- Тексты преобразуются в вектора TF-IDF, где каждый элемент вектора представляет собой вес термина в тексте, отражающий его важность в контексте корпуса документов.
- Вычисляется косинусное сходство между векторами TF-IDF.

**Второй метод**:
- Использует предварительно обученную модель BERT для векторизации текстов.
- Тексты представляются в виде последовательности токенов и затем подаются на вход модели BERT.
- Векторные представления получаются из последнего скрытого состояния модели BERT.
- Вычисляется косинусное сходство между векторами представлений текстов.

Оба метода в конечном итоге позволяют оценить схожесть между текстами, но подходы к векторизации их различаются. Первый метод использует контекстуальные представления слов с помощью модели BERT, что может учитывать семантические и контекстуальные аспекты текста, в то время как второй метод использует TF-IDF, который учитывает частоту встречаемости слов в тексте и в корпусе документов, но не учитывает контекст или семантику.


### Оценка с помощью sklearn (cos-sim-TF-IDF)

In [13]:
# 1 метод cos-sim-TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity_TF_IDF(row):
    documents = [row['answer'], row['ground_truth']]
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(documents)
    similarity = cosine_similarity(matrix)
    return similarity[0][1]


# Применение функции к DataFrame
df['cos-sim-TF-IDF'] = df.apply(calculate_cosine_similarity_TF_IDF, axis=1)

### Оценка BERTScorer - только на Python 3.10.4

In [14]:
# 2 метод cos-sim-BertModel
# Оценка с BertModel - только на Python 3.10.4
from transformers import BertTokenizer, BertModel
import numpy as np


# Загрузка предварительно обученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Определение функции для вычисления косинусного сходства с использованием модели BERT
def calculate_cosine_similarity_BertModel(row):
    text1 = row['answer']
    text2 = row['ground_truth']
    
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
    
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)
    
    embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()
    
    similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))
    return similarity[0][0]

# Применение функции к DataFrame
df['cos-sim-BertModel'] = df.apply(calculate_cosine_similarity_BertModel, axis=1)

### Предобработка текста

Чтобы тексты были более схожими, необходимо провести их предварительную обработку: привести все к нижнему регистру, выполнить леммантизацию и удалить пунктуацию:

In [15]:
import ssl
from nltk.corpus import stopwords
import spacy as sp

# Деактивация проверки сертификатов в NLTK 
ssl._create_default_https_context = ssl._create_unverified_context
# Загрузка продвинутой модели spaCy
nlp = sp.load('ru_core_news_sm')
# Загрузка списка стоп-слов для русского языка
stop_words = set(stopwords.words("russian"))

### Оценка с помощью spacy (sim-spacy)

In [16]:
# sim-spacy
def calculate_similarity_spacy(row):

    doc1 = nlp(preprocess(nlp(row['answer'])))
    doc2 = nlp(preprocess(nlp(row['ground_truth'])))
    return doc1.similarity(doc2)

def preprocess(text):
    preprocessed_text = [token.lemma_.lower() for token in text if token.lemma_.lower() not in stop_words and token.is_alpha]
    return ' '.join(preprocessed_text)


# Применение функции к DataFrame
df['sim-spacy'] = df.apply(calculate_similarity_spacy, axis=1)

/var/folders/c9/8zxgwvj510zc_3b1s0jbh5180000gn/T/ipykernel_6848/3253660487.py:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return doc1.similarity(doc2)


In [17]:
df.head()

,question,contexts,ground_truth,answer,faithfulness,answer_relevancy,context_precision,context_relevancy,answer_similarity,bleu_score,cos-sim-TF-IDF,cos-sim-BertModel,sim-spacy
0,Можно ли не расписываться на карте?,[['После получения карты поставьте свою подпис...,При отсутствии подписи карта может быть не при...,"Да, можно не расписываться на карте при возвра...",NaN,0.950380,1.0,1.0,0.834871,0.0,0.043307,0.982064,0.883591
1,Мой друг попросил меня дать свою карту для опл...,"[['Никогда не принимайте рекомендации, советы,...","Не передавайте карту другим лицам, в т.ч. родс...","Вам следует быть осторожным, если ваш друг пр...",NaN,0.000000,1.0,1.0,0.899626,0.0,0.068005,0.984102,0.597106
2,Безопасно ли хранить пинкод в заметках в телеф...,[['Категорически запрещается записывать ПИН-ко...,"Да, однако не указывайте в названии заметки, ч...","Нет, не безопасно хранить ПИН-код в заметках в...",NaN,0.962787,1.0,0.0,0.890383,0.0,0.113593,0.951827,0.582823
3,Я потеряла карту,[[' Вы потеряли карту или ее украли\nПрежде вс...,"Если вы не уверены, что карту украли, не спеши...","Свяжитесь с банком, который установил банкомат...",NaN,0.805542,1.0,0.1,0.849157,0.0,0.000000,0.967204,0.863695
4,Почему при использовании карты важно не распис...,"[['Проверяйте правильность суммы, указанной на...",Важно проверять правильность суммы перед подпи...,При использовании карты важно не расписываться...,NaN,0.940811,1.0,1.0,0.919985,0.0,0.101249,0.976521,0.916104


In [18]:
df.to_csv(path2 + '/evaluete_base_rag_gpt.csv', index=False)